<a class="anchor" id="section-0"></a>
<img src="https://i.imgur.com/lTCPbz0.jpeg" alt='logo puc-sp' width='30%' align='left'>

### Pontifícia Universidade Católica de São Paulo (PUC-SP)

<h1 style=font-size:30px>TrendSpotter: O Poder das Palavras-chave</h1>


### Bacharelado em Ciência de Dados e Inteligência Artificial

#### Turma: CDIA21-MA

**Professor:** Eduardo Savino Gomes

### Integrantes

<table align="left" style=font-size:15px>
<thead>
  <tr>
    <th style="text-align:left;">Nome</th>
    <th style="text-align:left;">RA</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td style="text-align:left;">Heloisa Mariani Rodrigues</td>
    <td>RA00297685</td>
  </tr>
  <tr>
    <td style="text-align:left;">João Pedro Taves Araujo</td>
    <td>RA00297753</td>
    
  </tr>
  <tr>
    <td style="text-align:left;">Lucas Lopes Amorim</td>
    <td>RA00303799</td>
    
  </tr>
  <tr>
    <td style="text-align:left;">Lucca Cerf Costa</td>
    <td>RA00304770</td>
  </tr>
    <tr>
    <td style="text-align:left;">Simeon Chavdar Ivanov</td>
    <td>RA00297777</td>
  </tr>
</tbody>
</table>

## Rapando artigos

In [ ]:
###Exemplo de uso com alguns scrapers

from src.Extractors import CanalTechScraper, OlharDigitalScraper, TecMundoScraper, TudoCelularScraper

#criando lista de extratores e rodando todos em um loop
extractors = [CanalTechScraper(), OlharDigitalScraper(), TecMundoScraper(), TudoCelularScraper()]
for extractor in extractors:
    extractor.get_articles(save=True)

###Todos os dados serão salvos como json na pasta data

## Contar palavras de artigos

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col, desc, lit, explode
import glob


spark = SparkSession.builder \
    .master('local[*]') \
    .appName("WordCount") \
    .getOrCreate()

In [4]:
spark

In [5]:
# Obtendo dados dos artigos raspados e excluindo desse processamento os dados do twitter (rede social)
file_paths = glob.glob('./src/clean_data/*json')
file_paths.remove('./src/clean_data/Twitter_clean.json')

In [7]:
def count_words(file_path, top_n_words=30):
    """
    Conta a frequência de palavras de um arquivo
    json e seleciona as top_n_wordsmais frequentes.
    """
    file_name = file_path.split('/')[-1].split('.')[0]
    df = spark.read.json(file_path)
    df_words = df.withColumn("words", split(df.text, "\s+"))
    df_exploded = df_words.select(df_words.link, explode(df_words.words).alias("word"))
    word_counts = df_exploded.groupBy("word").count()
    word_counts_sorted = word_counts.orderBy(word_counts["count"].desc()).limit(top_n_words)
    site_word_counts = word_counts_sorted.withColumn("site", lit(file_name))
    return site_word_counts

In [8]:
dfs = []
for file_ in file_paths:
    file_name = file_.split('/')[-1].split('.')[0]
    print(f'Processando {file_name}')
    word_counts = count_words(file_)
    dfs.append(word_counts)

Processando CanalTech


Processando G1
Processando ITForum
Processando TecMundo
Processando TudoCelular


Unindo todos os DataFrames em um só

In [9]:
result_df = dfs[0]
for i in range(1, len(dfs)):
    result_df = result_df.union(dfs[i])

In [10]:
result_df.show()

+------------+-----+---------+
|        word|count|     site|
+------------+-----+---------+
|     AirPods|   84|CanalTech|
|       Apple|   50|CanalTech|
|         Pro|   45|CanalTech|
|           2|   28|CanalTech|
|      estojo|   22|CanalTech|
|      ouvido|   21|CanalTech|
|      iPhone|   20|CanalTech|
|carregamento|   20|CanalTech|
|       fones|   18|CanalTech|
|        fone|   17|CanalTech|
|     bateria|   16|CanalTech|
|        USBC|   14|CanalTech|
|        novo|   14|CanalTech|
|       Passo|   14|CanalTech|
|       Beats|   12|CanalTech|
|          TV|   12|CanalTech|
|      volume|   12|CanalTech|
|         Fit|   11|CanalTech|
|       áudio|   11|CanalTech|
|        deve|   11|CanalTech|
+------------+-----+---------+
only showing top 20 rows



In [11]:
# Reduza o número de partições para 1
result_df = result_df.coalesce(1)

In [ ]:
# Salvando em CSV
result_df.write.csv("./src/output/word_counts.csv", header=True, mode="overwrite")

## Raspando rede social

In [ ]:
from src.Extractors import SocialMediaScraper

# Obtendo dados do twitter
# é necessário logar na conta para que o site usado para o scraping seja carregado
twitter_scraper = SocialMediaScraper(login, senha)
twitter_scraper.get_tweets(save=True)

## Contando palavras de rede social

In [22]:
from pyspark.sql.functions import when, size
from pyspark.sql import functions as F

In [79]:
def count_words_social_media(file_path, top_n_words=30):
    """
    Conta a frequência de palavras de um arquivo
    json, seleciona as top_n_wordsmais frequentes e 
    """
    file_name = file_path.split('/')[-1].split('.')[0]
    df = spark.read.json(file_path)
    
    df = df.withColumn("sentiment", when(col("sentiment") == "Positive", 1)
                                .when(col("sentiment") == "Neutral", 0)
                                .when(col("sentiment") == "Negative", -1))
    df_words = df.withColumn("words", split(df.text, "\s+"))
    df_exploded = df_words.select("sentiment", F.explode("words").alias("word"))

    word_counts = df_exploded.groupBy("word").count()
    sentiment_avg = df_exploded.groupBy("word").agg(F.avg("sentiment").alias("sentiment_avg"))
    df_final = word_counts.join(sentiment_avg, "word", "inner")
    df_final_sorted = df_final.orderBy(df_final["count"].desc()).limit(30)

    return df_final_sorted

In [ ]:
social_net_path = './src/clean_data/Twitter_clean.json'

In [80]:
twitter_data = count_words_social_media(social_net_path)

In [81]:
twitter_data.show()

+-------+-----+-------------------+
|   word|count|      sentiment_avg|
+-------+-----+-------------------+
|AirPods|   16|             -0.375|
|airpods|   10|               -0.1|
|    Pro|    9|-0.3333333333333333|
|      2|    6|-0.3333333333333333|
|    pra|    5|                0.0|
|  Apple|    5|               -0.2|
|   fone|    4|              -0.75|
|comprar|    3|               -1.0|
|      É|    3|-0.3333333333333333|
| ouvido|    3|-0.6666666666666666|
|      R|    3|                0.0|
|Airpods|    3| 0.6666666666666666|
|  perdi|    3|               -1.0|
|  Dolby|    2|               -1.0|
|      O|    2|                0.0|
|  preto|    2|               -0.5|
|    max|    2|                0.0|
|   vida|    2|                1.0|
|    pro|    2|                0.0|
| chique|    2|               -1.0|
+-------+-----+-------------------+
only showing top 20 rows



## Recomendamos um site para anunciar

- Pensando na utilização do nosso serviço em um cenário real de Big Data, pensamos em utilizar ferramentas como o ChatGPT para analisar os dados coletados e realizar a recomendação dos ssites para nós.

- O modelo utilizou um cálculo de similaridade de palavras, cruzando os dados de redes sociais com os dos sites para embasar a recomendação:

| Site        | Contagem de Palavras Correspondentes | Similaridade |
|-------------|--------------------------------------|--------------|
| CanalTech   | 84 + 24 + 17 + 3 = 128                                  | 128/87 = 1.47         |
| G1          | 24 + 17 + 4 + 3 + 5 + 5 + 16 + 9 + 3 + 3 = 89                                   |89/87 = 1.02         |
| ITForum     | 91                                   | 91/87 = 1.05         |
| TecMundo    | 145 + 81 + 59 + 16 + 50 + 23 + 13 + 18 =  405                                  | 405/87 = 4.66         |
| TudoCelular | 16 + 122 + 9 + 6 + 29 + 18 + 3 =  203                                  | 203/87 = 2.33         |


Portanto, fica claro que com os dados que temos, TecMundo seria o site mais favorável à indicação.

Ao pedir que o modelo elaborasse mais na recomendação, conseguimos o seguinte resultado:

1. Similaridade de conteúdo: O TecMundo obteve a maior pontuação de similaridade com o conteúdo sendo compartilhado sobre os AirPods no Twitter. Isso indica que o site está abordando tópicos relacionados aos AirPods de maneira mais abrangente e em linha com as conversas nas redes sociais. Ao anunciar nesse site, há uma maior probabilidade de atingir o público interessado no produto.

2. Audiência especializada: O TecMundo é conhecido por seu foco em tecnologia e por atrair uma audiência especializada nesse assunto. Ao anunciar os AirPods em um site voltado para tecnologia, é mais provável que você alcance pessoas que estejam ativamente buscando informações e avaliações sobre produtos desse segmento. Isso aumenta as chances de atingir consumidores interessados nos AirPods e potenciais compradores.

3. Relevância e autoridade: O TecMundo é reconhecido como uma fonte confiável de informações sobre tecnologia. Ao associar sua marca e produtos a um site respeitado e confiável, você pode aumentar a percepção de qualidade e confiança dos seus produtos. Além disso, o TecMundo possui uma base de leitores fiéis que acompanham regularmente o conteúdo do site, o que significa que seus anúncios terão visibilidade entre um público engajado.

4. Alcance e visibilidade: O TecMundo é um dos sites de tecnologia mais populares do Brasil, alcançando um grande número de leitores e seguidores. Ao anunciar os AirPods nesse site, você terá a oportunidade de alcançar uma ampla audiência interessada em tecnologia, incluindo potenciais compradores que estão em busca de informações sobre fones de ouvido sem fio.

Em resumo, escolher o TecMundo para posicionar os anúncios dos AirPods oferece a vantagem de estar em um site com conteúdo alinhado ao interesse do público, uma audiência especializada em tecnologia, uma fonte confiável de informações e um grande alcance para aumentar a visibilidade do produto. Esses fatores combinados podem contribuir para o sucesso da campanha de marketing e para alcançar o público certo para os AirPods.

## Apresentando resultados em um Dashboard do Tableau

<p align="center">
    <a href="https://public.tableau.com/views/WordCount_16871324744110/Dashboard"><img alt="Nuvem de Palavras" width="75%" src="img/dashboard.png" /></a>
</p>